In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.models import Sequential

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
import os
os.chdir('/kaggle/input')

In [ ]:
batch_size = 128

train_datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range=10.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    'dataset/train/',
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=batch_size,
    classes=['zero', 'one', 'two', 'three', 'four', 'five', 'ok', 'up', 'down'],
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    'dataset/test/',
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=batch_size,
    classes=['zero', 'one', 'two', 'three', 'four', 'five', 'ok', 'up', 'down'],
    class_mode='categorical'
)

In [ ]:
h1, _, _ = plt.hist(
    train_gen.classes, 
    bins=range(10), 
    alpha=.8, 
    color='blue', 
    edgecolor='black'
)
h2, _, _ = plt.hist(
    test_gen.classes, 
    bins=range(10), 
    alpha=.8, 
    color='red', 
    edgecolor='black'
)
plt.ylabel('Number of instances')
plt.xlabel('Class')

In [ ]:
X, y = train_gen.next()
print(X.shape, y.shape)

plt.figure(figsize=(16,16))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.axis('off')
    plt.title('Label: %d' % np.argmax(y[i]))
    img = np.uint8(255*X[i, :, :, 0])
    plt.imshow(img, cmap='gray')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=120,
    epochs=40,
    validation_data=test_gen,
    validation_steps=28,
    callbacks=None
)

In [ ]:
model.save('/kaggle/working/hand_detection_model_2.h5')